In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import pandas_profiling as pp
import joblib
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
#from lightgbm import LGBMClassifier
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV

In [4]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = "gdrive/My Drive/JPMC_Hackathon/"
data_path = root_path + "data/"




Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
df = pd.read_csv(data_path+"df_imputed.csv")



In [6]:
df = df.drop(['Unnamed: 0'],axis=1)


In [7]:
df

,GeoAreaName,Time_Detail,BOTHSEX.SKILL_READ.GRAD23,BOTHSEX.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.GRAD23,FEMALE.SKILL_MATH.LOWSEC,MALE.SKILL_MATH.GRAD23,BOTHSEX.SKILL_MATH.PRIMAR,MALE.SKILL_READ.GRAD23,MALE.SKILL_MATH.LOWSEC,FEMALE.SKILL_READ.LOWSEC,BOTHSEX.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.GRAD23,BOTHSEX.SKILL_MATH.LOWSEC,MALE.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.PRIMAR,BOTHSEX.SKILL_MATH.GRAD23,MALE.SKILL_MATH.PRIMAR,MALE.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.PRIMAR
0,Afghanistan,2016,22.000000,38.044833,22.500000,28.415389,26.500000,3.864781,21.500000,28.342292,41.673333,11.694222,22.500000,28.401915,14.557123,17.579252,24.500000,17.340764,34.876667,14.478434
1,Argentina,2006,71.727822,42.136034,57.444123,34.000000,56.624427,48.594414,69.282292,37.940000,50.180000,44.703388,75.794225,35.856520,42.086773,47.375020,56.771681,50.830232,33.100000,47.933275
2,Argentina,2013,76.323888,59.694245,73.766024,40.260000,68.402758,55.602812,72.491678,44.290000,55.436667,46.421038,79.997727,42.204386,41.369005,51.402342,71.147119,58.627596,38.446667,52.527277
3,Australia,2011,92.932800,80.290075,69.124687,80.610000,71.207350,62.941286,91.064702,80.133333,87.456667,67.081458,94.874844,80.366542,66.450352,67.986424,70.178897,64.269272,73.300000,61.604058
4,Australia,2016,94.486205,82.256701,55.635023,79.193333,60.686105,68.076427,92.935186,79.080000,89.810000,70.972600,96.233288,79.127074,68.692258,73.459034,58.198134,67.379992,74.693333,68.819378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,United States of America,2011,97.517359,79.042362,80.017767,81.593333,81.949313,70.987798,96.905010,83.320000,85.886667,70.972600,98.118761,82.451027,68.692258,73.459034,80.970278,71.663475,72.096667,70.300225
170,United States of America,2016,96.094080,80.355802,80.392980,83.505819,81.210919,68.076427,95.434808,83.669406,86.946667,70.972600,96.831839,83.568997,68.692258,73.459034,80.823697,67.379992,73.686667,68.819378
171,Burkina Faso,2014,35.590000,38.044833,56.790000,35.030000,61.040000,21.900000,30.695329,44.710000,41.673333,21.400000,39.723117,39.779841,22.130000,19.580000,58.880000,24.120000,34.876667,17.900000
172,Uruguay,2006,75.350014,53.362590,69.059112,52.000000,67.870821,72.717498,73.190976,55.870000,61.090000,59.126344,79.883974,53.886323,54.859183,63.286937,68.265885,73.882887,45.260000,73.388436


# Data Augmentation
## Step 1. Linear Interpolation (not ideal but while we wait for other data to process this is what we will do to run our models)

In [8]:
X = df.drop(['GeoAreaName','Time_Detail'],axis=1)


,BOTHSEX.SKILL_READ.GRAD23,BOTHSEX.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.GRAD23,FEMALE.SKILL_MATH.LOWSEC,MALE.SKILL_MATH.GRAD23,BOTHSEX.SKILL_MATH.PRIMAR,MALE.SKILL_READ.GRAD23,MALE.SKILL_MATH.LOWSEC,FEMALE.SKILL_READ.LOWSEC,BOTHSEX.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.GRAD23,BOTHSEX.SKILL_MATH.LOWSEC,MALE.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.PRIMAR,BOTHSEX.SKILL_MATH.GRAD23,MALE.SKILL_MATH.PRIMAR,MALE.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.PRIMAR
0,22.000000,38.044833,22.500000,28.415389,26.500000,3.864781,21.500000,28.342292,41.673333,11.694222,22.500000,28.401915,14.557123,17.579252,24.500000,17.340764,34.876667,14.478434
1,71.727822,42.136034,57.444123,34.000000,56.624427,48.594414,69.282292,37.940000,50.180000,44.703388,75.794225,35.856520,42.086773,47.375020,56.771681,50.830232,33.100000,47.933275
2,76.323888,59.694245,73.766024,40.260000,68.402758,55.602812,72.491678,44.290000,55.436667,46.421038,79.997727,42.204386,41.369005,51.402342,71.147119,58.627596,38.446667,52.527277
3,92.932800,80.290075,69.124687,80.610000,71.207350,62.941286,91.064702,80.133333,87.456667,67.081458,94.874844,80.366542,66.450352,67.986424,70.178897,64.269272,73.300000,61.604058
4,94.486205,82.256701,55.635023,79.193333,60.686105,68.076427,92.935186,79.080000,89.810000,70.972600,96.233288,79.127074,68.692258,73.459034,58.198134,67.379992,74.693333,68.819378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,97.517359,79.042362,80.017767,81.593333,81.949313,70.987798,96.905010,83.320000,85.886667,70.972600,98.118761,82.451027,68.692258,73.459034,80.970278,71.663475,72.096667,70.300225
170,96.094080,80.355802,80.392980,83.505819,81.210919,68.076427,95.434808,83.669406,86.946667,70.972600,96.831839,83.568997,68.692258,73.459034,80.823697,67.379992,73.686667,68.819378
171,35.590000,38.044833,56.790000,35.030000,61.040000,21.900000,30.695329,44.710000,41.673333,21.400000,39.723117,39.779841,22.130000,19.580000,58.880000,24.120000,34.876667,17.900000
172,75.350014,53.362590,69.059112,52.000000,67.870821,72.717498,73.190976,55.870000,61.090000,59.126344,79.883974,53.886323,54.859183,63.286937,68.265885,73.882887,45.260000,73.388436


In [9]:
dataColnames = X.columns

In [10]:
dataColnames

Index(['BOTHSEX.SKILL_READ.GRAD23', 'BOTHSEX.SKILL_READ.LOWSEC',
       'FEMALE.SKILL_MATH.GRAD23', 'FEMALE.SKILL_MATH.LOWSEC',
       'MALE.SKILL_MATH.GRAD23', 'BOTHSEX.SKILL_MATH.PRIMAR',
       'MALE.SKILL_READ.GRAD23', 'MALE.SKILL_MATH.LOWSEC',
       'FEMALE.SKILL_READ.LOWSEC', 'BOTHSEX.SKILL_READ.PRIMAR',
       'FEMALE.SKILL_READ.GRAD23', 'BOTHSEX.SKILL_MATH.LOWSEC',
       'MALE.SKILL_READ.PRIMAR', 'FEMALE.SKILL_READ.PRIMAR',
       'BOTHSEX.SKILL_MATH.GRAD23', 'MALE.SKILL_MATH.PRIMAR',
       'MALE.SKILL_READ.LOWSEC', 'FEMALE.SKILL_MATH.PRIMAR'],
      dtype='object')

NameError: ignored

,GeoAreaName,Time_Detail,BOTHSEX.SKILL_READ.GRAD23,BOTHSEX.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.GRAD23,FEMALE.SKILL_MATH.LOWSEC,MALE.SKILL_MATH.GRAD23,BOTHSEX.SKILL_MATH.PRIMAR,MALE.SKILL_READ.GRAD23,MALE.SKILL_MATH.LOWSEC,FEMALE.SKILL_READ.LOWSEC,BOTHSEX.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.GRAD23,BOTHSEX.SKILL_MATH.LOWSEC,MALE.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.PRIMAR,BOTHSEX.SKILL_MATH.GRAD23,MALE.SKILL_MATH.PRIMAR,MALE.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.PRIMAR


In [12]:
df_allCountryNewData = pd.DataFrame(columns=df.columns)

for country in df['GeoAreaName']:
  df_country = df[df['GeoAreaName']==country]
  if len(df_country) > 1:
    minYear = min(df_country['Time_Detail'])
    maxYear = max(df_country['Time_Detail'])
    print(country,minYear,maxYear)
    minYearVals = df_country[df_country['Time_Detail'] == minYear].drop(['GeoAreaName','Time_Detail'],axis=1).to_numpy()
    maxYearVals = df_country[df_country['Time_Detail'] == maxYear].drop(['GeoAreaName','Time_Detail'],axis=1).to_numpy()
    linearInterp = (maxYearVals-minYearVals) / (int(maxYear) - int(minYear))
    print(country,minYear,maxYear,linearInterp)
    baseYear2000Vals = minYearVals - (minYear - 2000) * linearInterp
    newYearDataList = []
    for yearScalar in range(20):
      d = pd.DataFrame.from_dict({'GeoAreaName':[country],"Time_Detail":[str(2000+yearScalar)]})
      newYearData = baseYear2000Vals + yearScalar * linearInterp
      df_tempData = pd.DataFrame(data=newYearData,columns=dataColnames)
      df_newYearData = pd.concat([d, df_tempData], axis=1)
      print(df_newYearData)
      df_allCountryNewData=df_allCountryNewData.append(df_newYearData) 
  
  # not using countries with only one data point
  #else:
  #  minYear = min(df_country['Time_Detail'])
  #  #maxYear = max(df_country['Time_Detail'])
  #  #print(country,minYear,maxYear)
  #  minYearVals = df_country[df_country['Time_Detail'] == minYear].drop(['GeoAreaName','Time_Detail'],axis=1).to_numpy()
  #  #maxYearVals = df_country[df_country['Time_Detail'] == maxYear].drop(['GeoAreaName','Time_Detail'],axis=1).to_numpy()
  #  for yearScalar in range(19):
  #    d = pd.DataFrame({'GeoAreaName':[country],"Time_Detail":[str(2000+yearScalar)]})
  #    newYearData = minYearVals
  #    df_tempData = pd.DataFrame(data=newYearData,columns=dataColnames)
  #    df_newYearData = pd.concat([d, df_tempData], axis=1)
  #    print(df_newYearData)
  #    df_allCountryNewData=df_allCountryNewData.append(df_newYearData) 
  


Streaming output truncated to the last 5000 lines.
   GeoAreaName Time_Detail  ...  MALE.SKILL_READ.LOWSEC  FEMALE.SKILL_MATH.PRIMAR
0  New Zealand        2016  ...                   70.39                 54.628319

[1 rows x 20 columns]
   GeoAreaName Time_Detail  ...  MALE.SKILL_READ.LOWSEC  FEMALE.SKILL_MATH.PRIMAR
0  New Zealand        2017  ...                   70.39                 54.628319

[1 rows x 20 columns]
   GeoAreaName Time_Detail  ...  MALE.SKILL_READ.LOWSEC  FEMALE.SKILL_MATH.PRIMAR
0  New Zealand        2018  ...                   70.39                 54.628319

[1 rows x 20 columns]
   GeoAreaName Time_Detail  ...  MALE.SKILL_READ.LOWSEC  FEMALE.SKILL_MATH.PRIMAR
0  New Zealand        2019  ...                   70.39                 54.628319

[1 rows x 20 columns]
New Zealand 2001 2016
New Zealand 2001 2016 [[-7.01716607e-04  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.69803127e-02  0.00000000e+00
   1.89478063e-15  0.0000

In [ ]:
df_allCountryNewData



,GeoAreaName,Time_Detail,BOTHSEX.SKILL_READ.GRAD23,BOTHSEX.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.GRAD23,FEMALE.SKILL_MATH.LOWSEC,MALE.SKILL_MATH.GRAD23,BOTHSEX.SKILL_MATH.PRIMAR,MALE.SKILL_READ.GRAD23,MALE.SKILL_MATH.LOWSEC,FEMALE.SKILL_READ.LOWSEC,BOTHSEX.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.GRAD23,BOTHSEX.SKILL_MATH.LOWSEC,MALE.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.PRIMAR,BOTHSEX.SKILL_MATH.GRAD23,MALE.SKILL_MATH.PRIMAR,MALE.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.PRIMAR
0,Afghanistan,2000,22.000000,38.044833,22.500000,28.415389,26.500000,3.864781,21.500000,28.342292,41.673333,11.694222,22.500000,28.401915,14.557123,17.579252,24.500000,17.340764,34.876667,14.478434
0,Afghanistan,2001,22.000000,38.044833,22.500000,28.415389,26.500000,3.864781,21.500000,28.342292,41.673333,11.694222,22.500000,28.401915,14.557123,17.579252,24.500000,17.340764,34.876667,14.478434
0,Afghanistan,2002,22.000000,38.044833,22.500000,28.415389,26.500000,3.864781,21.500000,28.342292,41.673333,11.694222,22.500000,28.401915,14.557123,17.579252,24.500000,17.340764,34.876667,14.478434
0,Afghanistan,2003,22.000000,38.044833,22.500000,28.415389,26.500000,3.864781,21.500000,28.342292,41.673333,11.694222,22.500000,28.401915,14.557123,17.579252,24.500000,17.340764,34.876667,14.478434
0,Afghanistan,2004,22.000000,38.044833,22.500000,28.415389,26.500000,3.864781,21.500000,28.342292,41.673333,11.694222,22.500000,28.401915,14.557123,17.579252,24.500000,17.340764,34.876667,14.478434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Uruguay,2015,81.052638,61.663313,79.152179,51.052857,75.111747,66.950077,79.292783,54.618571,62.440000,58.413828,82.340148,52.818471,53.635750,63.207486,77.074336,68.239972,44.904286,65.245313
0,Uruguay,2016,81.686262,62.585615,80.273631,50.947619,75.916294,66.309252,79.970762,54.479524,62.590000,58.334660,82.613057,52.699821,53.499813,63.198658,78.053052,67.612981,44.864762,64.340522
0,Uruguay,2017,82.319887,63.507918,81.395083,50.842381,76.720841,65.668428,80.648741,54.340476,62.740000,58.255491,82.885965,52.581170,53.363876,63.189830,79.031769,66.985991,44.825238,63.435730
0,Uruguay,2018,82.953512,64.430220,82.516535,50.737143,77.525389,65.027603,81.326719,54.201429,62.890000,58.176323,83.158873,52.462520,53.227939,63.181002,80.010486,66.359000,44.785714,62.530939


## Analyses we can do
1. Proportion of children and young people achieving a minimum proficiency level in reading and mathematics (%)
2. **Completion rate, by sex, location, wealth quintile and education level (%)
bold text**
3. Proportion of children aged 36−59 months who are developmentally on track in at least three of the following domains: literacy-numeracy, physical development, social-emotional development, and learning (% of children aged 36-59 months)
4. **Participation rate in organized learning (one year before the official primary entry age), by sex (%)**
5. **Participation rate in formal and non-formal education and training, by sex (%)**
6. Proportion of youth and adults with information and communications technology (ICT) skills, by sex and type of skill (%)
7. **Adjusted gender parity index for completion rate, by sex, location, wealth quintile and education level**
8. Adjusted location parity index for completion rate, by sex, location, wealth quintile and education level
9. Adjusted wealth parity index for completion rate, by sex, location, wealth quintile and education level
10. Gender parity index for youth/adults with information and communications technology (ICT) skills, by type of skill (ratio)
11. **Gender parity index for participation rate in formal and non-formal education and training (ratio)**
12. **Gender parity index for participation rate in organized learning (one year before the official primary entry age), (ratio)**
13. **Gender parity index of trained teachers, by education level (ratio)**
14. Immigration status parity index for achieving at least a fixed level of proficiency in functional skills, by numeracy/literacy skills (ratio)
15. Language test parity index for achievement (ratio)
16. Native parity index for achievement (ratio)
17. **Gender parity index for achievement (ratio)**
18. Rural to urban parity index for achievement (ratio)
19. Low to high socio-economic parity status index for achievement (ratio)
20. Proportion of population achieving at least a fixed level of proficiency in functional skills, by sex, age and type of skill (%)
21. Schools with basic handwashing facilities, by education level (%)
22. Schools with access to\xa0computers for pedagogical purposes, by education level (%)
23. Schools with access to\xa0electricity, by education level (%)
24. Schools with access to basic drinking water, by education level (%)
25. Schools with access to the internet for pedagogical purposes, by education level (%)
26. Schools with access to\xa0access to single-sex basic sanitation, by education level (%)
27. Proportion of schools with access to adapted infrastructure and materials for students with disabilities, by education level (%)
28. Total official flows for scholarships, by recipient countries (millions of constant 2018 United States dollars)
29. Proportion of teachers who have received at least the minimum organized teacher training (e.g. pedagogical training) pre-service or in-service required for teaching at the relevant level in a given country, by sex and education level (%)



In [ ]:
df_allCountryNewData

,GeoAreaName,Time_Detail,BOTHSEX.SKILL_READ.GRAD23,BOTHSEX.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.GRAD23,FEMALE.SKILL_MATH.LOWSEC,MALE.SKILL_MATH.GRAD23,BOTHSEX.SKILL_MATH.PRIMAR,MALE.SKILL_READ.GRAD23,MALE.SKILL_MATH.LOWSEC,FEMALE.SKILL_READ.LOWSEC,BOTHSEX.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.GRAD23,BOTHSEX.SKILL_MATH.LOWSEC,MALE.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.PRIMAR,BOTHSEX.SKILL_MATH.GRAD23,MALE.SKILL_MATH.PRIMAR,MALE.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.PRIMAR
0,Argentina,2000,67.788337,27.086139,43.453923,28.634286,46.528714,42.587215,66.531389,32.497143,45.674286,43.231117,72.191223,30.415491,42.702003,43.923031,44.449877,44.146778,28.517143,43.995560
0,Argentina,2001,68.444918,29.594455,45.785623,29.528571,48.211333,43.588415,66.989873,33.404286,46.425238,43.476496,72.791723,31.322329,42.599464,44.498362,46.503511,45.260687,29.280952,44.651846
0,Argentina,2002,69.101498,32.102771,48.117323,30.422857,49.893952,44.589615,67.448357,34.311429,47.176190,43.721874,73.392223,32.229168,42.496926,45.073694,48.557145,46.374596,30.044762,45.308132
0,Argentina,2003,69.758079,34.611087,50.449023,31.317143,51.576571,45.590814,67.906840,35.218571,47.927143,43.967253,73.992724,33.136006,42.394388,45.649026,50.610779,47.488505,30.808571,45.964418
0,Argentina,2004,70.414660,37.119402,52.780723,32.211429,53.259189,46.592014,68.365324,36.125714,48.678095,44.212631,74.593224,34.042844,42.291849,46.224357,52.664413,48.602414,31.572381,46.620704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Uruguay,2015,81.052638,61.663313,79.152179,51.052857,75.111747,66.950077,79.292783,54.618571,62.440000,58.413828,82.340148,52.818471,53.635750,63.207486,77.074336,68.239972,44.904286,65.245313
0,Uruguay,2016,81.686262,62.585615,80.273631,50.947619,75.916294,66.309252,79.970762,54.479524,62.590000,58.334660,82.613057,52.699821,53.499813,63.198658,78.053052,67.612981,44.864762,64.340522
0,Uruguay,2017,82.319887,63.507918,81.395083,50.842381,76.720841,65.668428,80.648741,54.340476,62.740000,58.255491,82.885965,52.581170,53.363876,63.189830,79.031769,66.985991,44.825238,63.435730
0,Uruguay,2018,82.953512,64.430220,82.516535,50.737143,77.525389,65.027603,81.326719,54.201429,62.890000,58.176323,83.158873,52.462520,53.227939,63.181002,80.010486,66.359000,44.785714,62.530939


In [ ]:
df_allCountryNewData.columns

Index(['GeoAreaName', 'Time_Detail', 'BOTHSEX.SKILL_READ.GRAD23',
       'BOTHSEX.SKILL_READ.LOWSEC', 'FEMALE.SKILL_MATH.GRAD23',
       'FEMALE.SKILL_MATH.LOWSEC', 'MALE.SKILL_MATH.GRAD23',
       'BOTHSEX.SKILL_MATH.PRIMAR', 'MALE.SKILL_READ.GRAD23',
       'MALE.SKILL_MATH.LOWSEC', 'FEMALE.SKILL_READ.LOWSEC',
       'BOTHSEX.SKILL_READ.PRIMAR', 'FEMALE.SKILL_READ.GRAD23',
       'BOTHSEX.SKILL_MATH.LOWSEC', 'MALE.SKILL_READ.PRIMAR',
       'FEMALE.SKILL_READ.PRIMAR', 'BOTHSEX.SKILL_MATH.GRAD23',
       'MALE.SKILL_MATH.PRIMAR', 'MALE.SKILL_READ.LOWSEC',
       'FEMALE.SKILL_MATH.PRIMAR'],
      dtype='object')

In [39]:
#df_allCountryNewData.to_csv(data_path+"df_allCountryNewData.csv")
df_allCountryNewData = pd.read_csv(data_path+"df_allCountryNewData.csv")

In [ ]:
df_allCountryNewData

In [14]:
targetsList = ['BOTHSEX.SKILL_READ.GRAD23',
       'BOTHSEX.SKILL_READ.LOWSEC', 'FEMALE.SKILL_MATH.GRAD23',
       'FEMALE.SKILL_MATH.LOWSEC', 'MALE.SKILL_MATH.GRAD23',
       'BOTHSEX.SKILL_MATH.PRIMAR', 'MALE.SKILL_READ.GRAD23',
       'MALE.SKILL_MATH.LOWSEC', 'FEMALE.SKILL_READ.LOWSEC',
       'BOTHSEX.SKILL_READ.PRIMAR', 'FEMALE.SKILL_READ.GRAD23',
       'BOTHSEX.SKILL_MATH.LOWSEC', 'MALE.SKILL_READ.PRIMAR',
       'FEMALE.SKILL_READ.PRIMAR', 'BOTHSEX.SKILL_MATH.GRAD23',
       'MALE.SKILL_MATH.PRIMAR', 'MALE.SKILL_READ.LOWSEC',
       'FEMALE.SKILL_MATH.PRIMAR']

In [ ]:
targetsList

['BOTHSEX.SKILL_READ.GRAD23',
 'BOTHSEX.SKILL_READ.LOWSEC',
 'FEMALE.SKILL_MATH.GRAD23',
 'FEMALE.SKILL_MATH.LOWSEC',
 'MALE.SKILL_MATH.GRAD23',
 'BOTHSEX.SKILL_MATH.PRIMAR',
 'MALE.SKILL_READ.GRAD23',
 'MALE.SKILL_MATH.LOWSEC',
 'FEMALE.SKILL_READ.LOWSEC',
 'BOTHSEX.SKILL_READ.PRIMAR',
 'FEMALE.SKILL_READ.GRAD23',
 'BOTHSEX.SKILL_MATH.LOWSEC',
 'MALE.SKILL_READ.PRIMAR',
 'FEMALE.SKILL_READ.PRIMAR',
 'BOTHSEX.SKILL_MATH.GRAD23',
 'MALE.SKILL_MATH.PRIMAR',
 'MALE.SKILL_READ.LOWSEC',
 'FEMALE.SKILL_MATH.PRIMAR']

In [ ]:
df_allCountryNewData['GeoAreaName'].unique()

array(['Argentina', 'Australia', 'Bangladesh', 'Brazil', 'Canada',
       'Chile', 'Colombia', 'Costa Rica', 'Czechia', 'Denmark',
       'Dominican Republic', 'Ecuador', 'Finland', 'Georgia', 'Guatemala',
       'China, Hong Kong Special Administrative Region', 'Iceland',
       'Indonesia', 'Iran (Islamic Republic of)', 'Ireland', 'Israel',
       'Italy', 'Kenya', 'Kuwait', 'Latvia', 'Malta', 'Mexico', 'Morocco',
       'Oman', 'Netherlands', 'New Zealand', 'Nicaragua', 'Norway',
       'Panama', 'Paraguay', 'Peru', 'Poland', 'Portugal', 'Qatar',
       'Romania', 'Saudi Arabia', 'Singapore', 'Slovenia', 'South Africa',
       'Spain', 'Sweden', 'Trinidad and Tobago', 'United Arab Emirates',
       'Uganda', 'North Macedonia', 'United States of America', 'Uruguay'],
      dtype=object)

In [ ]:
df_allCountryNewData

,GeoAreaName,Time_Detail,BOTHSEX.SKILL_READ.GRAD23,BOTHSEX.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.GRAD23,FEMALE.SKILL_MATH.LOWSEC,MALE.SKILL_MATH.GRAD23,BOTHSEX.SKILL_MATH.PRIMAR,MALE.SKILL_READ.GRAD23,MALE.SKILL_MATH.LOWSEC,FEMALE.SKILL_READ.LOWSEC,BOTHSEX.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.GRAD23,BOTHSEX.SKILL_MATH.LOWSEC,MALE.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.PRIMAR,BOTHSEX.SKILL_MATH.GRAD23,MALE.SKILL_MATH.PRIMAR,MALE.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.PRIMAR
0,Argentina,2000,67.788337,27.086139,43.453923,28.634286,46.528714,42.587215,66.531389,32.497143,45.674286,43.231117,72.191223,30.415491,42.702003,43.923031,44.449877,44.146778,28.517143,43.995560
0,Argentina,2001,68.444918,29.594455,45.785623,29.528571,48.211333,43.588415,66.989873,33.404286,46.425238,43.476496,72.791723,31.322329,42.599464,44.498362,46.503511,45.260687,29.280952,44.651846
0,Argentina,2002,69.101498,32.102771,48.117323,30.422857,49.893952,44.589615,67.448357,34.311429,47.176190,43.721874,73.392223,32.229168,42.496926,45.073694,48.557145,46.374596,30.044762,45.308132
0,Argentina,2003,69.758079,34.611087,50.449023,31.317143,51.576571,45.590814,67.906840,35.218571,47.927143,43.967253,73.992724,33.136006,42.394388,45.649026,50.610779,47.488505,30.808571,45.964418
0,Argentina,2004,70.414660,37.119402,52.780723,32.211429,53.259189,46.592014,68.365324,36.125714,48.678095,44.212631,74.593224,34.042844,42.291849,46.224357,52.664413,48.602414,31.572381,46.620704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Uruguay,2015,81.052638,61.663313,79.152179,51.052857,75.111747,66.950077,79.292783,54.618571,62.440000,58.413828,82.340148,52.818471,53.635750,63.207486,77.074336,68.239972,44.904286,65.245313
0,Uruguay,2016,81.686262,62.585615,80.273631,50.947619,75.916294,66.309252,79.970762,54.479524,62.590000,58.334660,82.613057,52.699821,53.499813,63.198658,78.053052,67.612981,44.864762,64.340522
0,Uruguay,2017,82.319887,63.507918,81.395083,50.842381,76.720841,65.668428,80.648741,54.340476,62.740000,58.255491,82.885965,52.581170,53.363876,63.189830,79.031769,66.985991,44.825238,63.435730
0,Uruguay,2018,82.953512,64.430220,82.516535,50.737143,77.525389,65.027603,81.326719,54.201429,62.890000,58.176323,83.158873,52.462520,53.227939,63.181002,80.010486,66.359000,44.785714,62.530939


In [40]:
df_Colombia = df_allCountryNewData[df_allCountryNewData['GeoAreaName']=='Colombia'].drop(['GeoAreaName','Time_Detail'],axis=1)[:19]

df_CostaRica = df_allCountryNewData[df_allCountryNewData['GeoAreaName']=='Costa Rica'].drop(['GeoAreaName','Time_Detail'],axis=1)[:19]

df_Guatemala = df_allCountryNewData[df_allCountryNewData['GeoAreaName']=='Guatemala'].drop(['GeoAreaName','Time_Detail'],axis=1)[:19]

df_Mexico = df_allCountryNewData[df_allCountryNewData['GeoAreaName']=='Mexico'].drop(['GeoAreaName','Time_Detail'],axis=1)[:19]



In [41]:
df_Colombia  

,Unnamed: 0,BOTHSEX.SKILL_READ.GRAD23,BOTHSEX.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.GRAD23,FEMALE.SKILL_MATH.LOWSEC,MALE.SKILL_MATH.GRAD23,BOTHSEX.SKILL_MATH.PRIMAR,MALE.SKILL_READ.GRAD23,MALE.SKILL_MATH.LOWSEC,FEMALE.SKILL_READ.LOWSEC,BOTHSEX.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.GRAD23,BOTHSEX.SKILL_MATH.LOWSEC,MALE.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.PRIMAR,BOTHSEX.SKILL_MATH.GRAD23,MALE.SKILL_MATH.PRIMAR,MALE.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.PRIMAR
260,0,64.995083,28.028368,40.290258,2.742857,46.054428,29.812171,64.956083,17.994286,33.80,42.120373,66.555973,9.683807,44.836443,39.458300,42.832775,33.743060,36.757143,27.851154
261,0,66.071775,30.739886,42.173513,6.199048,47.449413,31.185868,65.828265,20.578571,35.92,43.132171,67.713828,12.753807,45.548822,40.780665,44.498433,35.188009,37.580952,28.945644
262,0,67.148466,33.451403,44.056768,9.655238,48.844398,32.559564,66.700448,23.162857,38.04,44.143968,68.871683,15.823806,46.261201,42.103030,46.164091,36.632959,38.404762,30.040134
263,0,68.225157,36.162921,45.940023,13.111429,50.239383,33.933260,67.572631,25.747143,40.16,45.155766,70.029539,18.893806,46.973580,43.425395,47.829749,38.077908,39.228571,31.134624
264,0,69.301849,38.874439,47.823277,16.567619,51.634368,35.306956,68.444814,28.331429,42.28,46.167564,71.187394,21.963805,47.685959,44.747760,49.495408,39.522857,40.052381,32.229114
265,0,70.378540,41.585956,49.706532,20.023810,53.029353,36.680652,69.316997,30.915714,44.40,47.179361,72.345249,25.033805,48.398338,46.070125,51.161066,40.967806,40.876190,33.323603
266,0,71.455231,44.297474,51.589787,23.480000,54.424337,38.054349,70.189180,33.500000,46.52,48.191159,73.503104,28.103805,49.110717,47.392490,52.826724,42.412756,41.700000,34.418093
267,0,72.531923,47.008991,53.473042,26.936190,55.819322,39.428045,71.061362,36.084286,48.64,49.202956,74.660959,31.173804,49.823096,48.714855,54.492382,43.857705,42.523810,35.512583
268,0,73.608614,49.720509,55.356297,30.392381,57.214307,40.801741,71.933545,38.668571,50.76,50.214754,75.818814,34.243804,50.535475,50.037220,56.158040,45.302654,43.347619,36.607073
269,0,74.685305,52.432027,57.239551,33.848571,58.609292,42.175437,72.805728,41.252857,52.88,51.226552,76.976669,37.313804,51.247854,51.359586,57.823698,46.747603,44.171429,37.701563


In [42]:
df_CostaRica

,Unnamed: 0,BOTHSEX.SKILL_READ.GRAD23,BOTHSEX.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.GRAD23,FEMALE.SKILL_MATH.LOWSEC,MALE.SKILL_MATH.GRAD23,BOTHSEX.SKILL_MATH.PRIMAR,MALE.SKILL_READ.GRAD23,MALE.SKILL_MATH.LOWSEC,FEMALE.SKILL_READ.LOWSEC,BOTHSEX.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.GRAD23,BOTHSEX.SKILL_MATH.LOWSEC,MALE.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.PRIMAR,BOTHSEX.SKILL_MATH.GRAD23,MALE.SKILL_MATH.PRIMAR,MALE.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.PRIMAR
300,0,87.163592,74.719121,59.942409,73.256667,67.130888,64.845852,87.305138,74.59,81.37,73.926149,87.413787,73.917305,74.697722,72.754159,63.155555,70.261515,68.09,58.729336
301,0,87.326207,74.719121,61.790421,73.256667,68.484430,64.480127,87.312589,74.59,81.37,73.493010,87.712476,73.917305,74.117849,72.507632,64.786794,69.718940,68.09,58.582828
302,0,87.488823,74.719121,63.638434,73.256667,69.837971,64.114402,87.320040,74.59,81.37,73.059872,88.011165,73.917305,73.537975,72.261105,66.418034,69.176365,68.09,58.436320
303,0,87.651438,74.719121,65.486447,73.256667,71.191513,63.748677,87.327491,74.59,81.37,72.626733,88.309854,73.917305,72.958102,72.014578,68.049274,68.633790,68.09,58.289811
304,0,87.814054,74.719121,67.334460,73.256667,72.545054,63.382953,87.334942,74.59,81.37,72.193595,88.608543,73.917305,72.378228,71.768051,69.680513,68.091214,68.09,58.143303
305,0,87.976669,74.719121,69.182473,73.256667,73.898596,63.017228,87.342393,74.59,81.37,71.760456,88.907233,73.917305,71.798355,71.521524,71.311753,67.548639,68.09,57.996795
306,0,88.139284,74.719121,71.030486,73.256667,75.252137,62.651503,87.349844,74.59,81.37,71.327318,89.205922,73.917305,71.218481,71.274997,72.942993,67.006064,68.09,57.850287
307,0,88.301900,74.719121,72.878499,73.256667,76.605679,62.285778,87.357295,74.59,81.37,70.894179,89.504611,73.917305,70.638608,71.028470,74.574232,66.463489,68.09,57.703779
308,0,88.464515,74.719121,74.726512,73.256667,77.959220,61.920053,87.364746,74.59,81.37,70.461041,89.803300,73.917305,70.058734,70.781943,76.205472,65.920914,68.09,57.557270
309,0,88.627131,74.719121,76.574525,73.256667,79.312762,61.554329,87.372197,74.59,81.37,70.027903,90.101990,73.917305,69.478861,70.535416,77.836712,65.378338,68.09,57.410762


In [43]:
df_Guatemala

,Unnamed: 0,BOTHSEX.SKILL_READ.GRAD23,BOTHSEX.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.GRAD23,FEMALE.SKILL_MATH.LOWSEC,MALE.SKILL_MATH.GRAD23,BOTHSEX.SKILL_MATH.PRIMAR,MALE.SKILL_READ.GRAD23,MALE.SKILL_MATH.LOWSEC,FEMALE.SKILL_READ.LOWSEC,BOTHSEX.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.GRAD23,BOTHSEX.SKILL_MATH.LOWSEC,MALE.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.PRIMAR,BOTHSEX.SKILL_MATH.GRAD23,MALE.SKILL_MATH.PRIMAR,MALE.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.PRIMAR
640,0,20.509883,33.246075,15.115299,40.126893,18.370891,10.354298,20.100844,40.199826,35.456190,7.304286,22.659952,40.511397,7.988308,7.789946,16.160154,10.256601,32.542381,11.255307
641,0,24.166269,34.045868,17.808108,39.137901,21.061996,12.208787,23.754968,39.737788,36.492381,9.540539,26.186194,39.739762,10.239700,9.921266,18.900588,12.473431,32.931429,12.677568
642,0,27.822656,34.845661,20.500918,38.148909,23.753100,14.063276,27.409093,39.275750,37.528571,11.776792,29.712437,38.968126,12.491092,12.052587,21.641023,14.690262,33.320476,14.099828
643,0,31.479042,35.645454,23.193727,37.159917,26.444204,15.917765,31.063218,38.813712,38.564762,14.013046,33.238679,38.196490,14.742484,14.183908,24.381457,16.907092,33.709524,15.522089
644,0,35.135428,36.445247,25.886537,36.170926,29.135308,17.772254,34.717342,38.351674,39.600952,16.249299,36.764922,37.424854,16.993876,16.315229,27.121891,19.123922,34.098571,16.944349
645,0,38.791814,37.245040,28.579346,35.181934,31.826412,19.626743,38.371467,37.889636,40.637143,18.485552,40.291164,36.653218,19.245268,18.446550,29.862325,21.340753,34.487619,18.366609
646,0,42.448200,38.044833,31.272155,34.192942,34.517517,21.481232,42.025592,37.427599,41.673333,20.721805,43.817406,35.881583,21.496660,20.577871,32.602760,23.557583,34.876667,19.788870
647,0,46.104587,38.844627,33.964965,33.203950,37.208621,23.335722,45.679716,36.965561,42.709524,22.958058,47.343649,35.109947,23.748051,22.709192,35.343194,25.774414,35.265714,21.211130
648,0,49.760973,39.644420,36.657774,32.214959,39.899725,25.190211,49.333841,36.503523,43.745714,25.194312,50.869891,34.338311,25.999443,24.840513,38.083628,27.991244,35.654762,22.633391
649,0,53.417359,40.444213,39.350584,31.225967,42.590829,27.044700,52.987966,36.041485,44.781905,27.430565,54.396134,33.566675,28.250835,26.971834,40.824062,30.208075,36.043810,24.055651


In [44]:
df_Mexico

,Unnamed: 0,BOTHSEX.SKILL_READ.GRAD23,BOTHSEX.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.GRAD23,FEMALE.SKILL_MATH.LOWSEC,MALE.SKILL_MATH.GRAD23,BOTHSEX.SKILL_MATH.PRIMAR,MALE.SKILL_READ.GRAD23,MALE.SKILL_MATH.LOWSEC,FEMALE.SKILL_READ.LOWSEC,BOTHSEX.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.GRAD23,BOTHSEX.SKILL_MATH.LOWSEC,MALE.SKILL_READ.PRIMAR,FEMALE.SKILL_READ.PRIMAR,BOTHSEX.SKILL_MATH.GRAD23,MALE.SKILL_MATH.PRIMAR,MALE.SKILL_READ.LOWSEC,FEMALE.SKILL_MATH.PRIMAR
1320,0,75.795616,47.135339,57.969777,36.918571,57.732310,49.473345,74.991969,42.250000,60.144286,51.260133,80.137943,39.459925,47.867381,54.856061,56.584863,46.202837,48.647143,53.137275
1321,0,75.947949,48.110983,59.513893,37.637143,59.130392,51.010986,75.021963,42.866667,60.061905,51.740892,80.140960,40.133340,48.400779,55.279414,58.153768,48.007209,48.174286,54.377718
1322,0,76.100282,49.086626,61.058009,38.355714,60.528474,52.548627,75.051958,43.483333,59.979524,52.221651,80.143977,40.806755,48.934176,55.702768,59.722674,49.811581,47.701429,55.618162
1323,0,76.252615,50.062270,62.602125,39.074286,61.926556,54.086268,75.081952,44.100000,59.897143,52.702410,80.146994,41.480170,49.467574,56.126121,61.291579,51.615953,47.228571,56.858605
1324,0,76.404948,51.037914,64.146241,39.792857,63.324638,55.623909,75.111946,44.716667,59.814762,53.183169,80.150010,42.153585,50.000972,56.549475,62.860485,53.420324,46.755714,58.099049
1325,0,76.557281,52.013558,65.690357,40.511429,64.722720,57.161550,75.141940,45.333333,59.732381,53.663928,80.153027,42.827000,50.534369,56.972828,64.429391,55.224696,46.282857,59.339492
1326,0,76.709614,52.989201,67.234473,41.230000,66.120802,58.699192,75.171935,45.950000,59.650000,54.144687,80.156044,43.500415,51.067767,57.396182,65.998296,57.029068,45.810000,60.579936
1327,0,76.861947,53.964845,68.778589,41.948571,67.518884,60.236833,75.201929,46.566667,59.567619,54.625446,80.159061,44.173830,51.601165,57.819535,67.567202,58.833440,45.337143,61.820379
1328,0,77.014279,54.940489,70.322705,42.667143,68.916966,61.774474,75.231923,47.183333,59.485238,55.106206,80.162077,44.847245,52.134562,58.242889,69.136107,60.637811,44.864286,63.060823
1329,0,77.166612,55.916133,71.866821,43.385714,70.315048,63.312115,75.261918,47.800000,59.402857,55.586965,80.165094,45.520660,52.667960,58.666242,70.705013,62.442183,44.391429,64.301266


NameError: ignored

In [45]:
df_Colombia.shape

(20, 19)

In [46]:
from statsmodels.tsa.vector_ar.var_model import VAR
# fit model
df_Colombia=df_Colombia.drop(df_Colombia.columns[0], axis=1)
data = df_Colombia.to_numpy()
model_Columbia = VAR(data)
model_fit_Columbia = model_Columbia.fit()
print(model_fit_Columbia.summary())
# make prediction
yhat_Columbia = model_fit_Columbia.forecast(model_fit_Columbia.y, steps=1)
print(yhat_Columbia)


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/vector_ar/var_model.py:711: RuntimeWarning: divide by zero encountered in true_divide
  omega = sse / df_resid
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/vector_ar/var_model.py:1349: RuntimeWarning: invalid value encountered in multiply
  return self.sigma_u * self.df_resid / self.nobs


ValueError: ignored

In [34]:
from statsmodels.tsa.vector_ar.var_model import VAR
# fit model
df_CostaRica=df_CostaRica.drop(df_CostaRica.columns[0], axis=1)
data = df_CostaRica.to_numpy()
model_CostaRica = VAR(data)
model_fit_CostaRica = model_CostaRica.fit()
print(model_fit_CostaRica.summary())
# make prediction
yhat_CostaRica = model_fit_CostaRica.forecast(model_fit_CostaRica.y, steps=1)
print(yhat_CostaRica)


ValueError: ignored

In [35]:
from statsmodels.tsa.vector_ar.var_model import VAR
# fit model
df_Guatemala=df_Guatemala.drop(df_Guatemala.columns[0], axis=1)
data = df_Guatemala.to_numpy()
model_Guatemala = VAR(data)
model_fit_Guatemala = model_Guatemala.fit()
print(model_fit_Guatemala.summary())
# make prediction
yhat_Guatemala = model_fit_Guatemala.forecast(model_fit_Guatemala.y, steps=1)
print(yhat_Guatemala)


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/vector_ar/var_model.py:711: RuntimeWarning: divide by zero encountered in true_divide
  omega = sse / df_resid
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/vector_ar/var_model.py:1349: RuntimeWarning: invalid value encountered in multiply
  return self.sigma_u * self.df_resid / self.nobs


ValueError: ignored

In [36]:
from statsmodels.tsa.vector_ar.var_model import VAR
# fit model
df_Mexico=df_Mexico.drop(df_Mexico.columns[0], axis=1)
data = df_Mexico.to_numpy()
model_Mexico = VAR(data)
model_fit_Mexico = model_Mexico.fit()
print(model_fit_Mexico.summary())
# make prediction
yhat_Mexico = model_fit_Mexico.forecast(model_fit_Mexico.y, steps=1)
print(yhat_Mexico)



/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/vector_ar/var_model.py:711: RuntimeWarning: divide by zero encountered in true_divide
  omega = sse / df_resid
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/vector_ar/var_model.py:1349: RuntimeWarning: invalid value encountered in multiply
  return self.sigma_u * self.df_resid / self.nobs


ValueError: ignored

In [ ]:
x

,Goal,Target,Indicator,SeriesCode,SeriesDescription,GeoAreaCode,GeoAreaName,TimePeriod,Time_Detail,TimeCoverage,...,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51
0,4,4.1,4.1.1,SE_TOT_PRFL,Proportion of children and young people achiev...,4,Afghanistan,2013,2013,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,4.1,4.1.1,SE_TOT_PRFL,Proportion of children and young people achiev...,4,Afghanistan,2013,2013,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,4.1,4.1.1,SE_TOT_PRFL,Proportion of children and young people achiev...,4,Afghanistan,2016,2016,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4.1,4.1.1,SE_TOT_PRFL,Proportion of children and young people achiev...,4,Afghanistan,2016,2016,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4.1,4.1.1,SE_TOT_PRFL,Proportion of children and young people achiev...,4,Afghanistan,2016,2016,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191757,4,4.c,4.c.1,SE_TRA_GRDL,Proportion of teachers who have received at le...,894,Zambia,2012,2012,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191758,4,4.c,4.c.1,SE_TRA_GRDL,Proportion of teachers who have received at le...,894,Zambia,2012,2012,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191759,4,4.c,4.c.1,SE_TRA_GRDL,Proportion of teachers who have received at le...,894,Zambia,2017,2017,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191760,4,4.c,4.c.1,SE_TRA_GRDL,Proportion of teachers who have received at le...,894,Zambia,2017,2017,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
